In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as data

from src.datasets.huge_stock_ds import spy
import importlib
importlib.reload(spy)

%matplotlib inline

data_dir = "../data/kaggle-huge-stock-market-dataset"
etfs = os.path.join(data_dir, "ETFs")
stocks = os.path.join(data_dir, "Stocks")

ds = spy.SPYDataset(data_dir, ['Open'], 'Close')

In [9]:
train_ds, test_ds = data.random_split(ds, [int(0.8*len(ds)), len(ds)-int(0.8*len(ds))])
train_dl = data.DataLoader(train_ds, batch_size=32, shuffle=True)

In [25]:
class BasicDnnRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(1, 32)
        self.layer2 = nn.Linear(32, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.05)
        self.layer3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        return x

In [26]:
net = BasicDnnRegression()
net = net.to(torch.device('cuda'))

In [30]:
from tqdm import tqdm

epochs = 10
optim = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.MSELoss()

for i in range(epochs):
    for x, y in (pbar := tqdm(train_dl, total=len(train_dl))):
        x = x.to(torch.device('cuda'))
        x = x.to(torch.float)
        y = y.to(torch.device('cuda'))
        y = y.to(torch.float)
        y = y.view(-1, 1)


        y_pred = net(x)
        loss_val = loss(y_pred, y)

        optim.zero_grad()
        loss_val.backward()
        optim.step()

        pbar.set_description(f"Epoch: {i + 1}, Loss: {loss_val.item():.4f}")

Loss: 29.9290: 100%|██████████| 80/80 [00:02<00:00, 39.90it/s] 


In [34]:
net.forward(torch.Tensor([436.94]).to('cuda'))

tensor([461.3300], device='cuda:0', grad_fn=<AddBackward0>)